In [70]:
import numpy as np 

z_hit = 0.85
z_short = 0.10
z_max = 0.025
z_rand = 0.025
lambda_short = 0.25
sigma_hit = 0.1
lidar_max = 2

zm=np.arange(0, lidar_max+1, 1)
ze=np.arange(0, lidar_max+1, 1)

ZM = zm.reshape(-1,+1)*np.ones(lidar_max+1).reshape(+1,-1)
ZE = ze.reshape(+1,-1)*np.ones(lidar_max+1).reshape(-1,+1)
Z = ZM-ZE

# P_hit: z_hit missing -> later 
P_hit = (np.sqrt(2*np.pi)*sigma_hit)*np.exp(-Z**2/(2*sigma_hit**2))
P_hit = P_hit/np.sum(P_hit, axis=0)
print(P_hit, np.sum(P_hit, axis=0))

# P_short: Z_short missing -> later
T_short = np.where(ZM>ZE, 0, 1)
P_short = lambda_short*np.exp(-lambda_short*ZM)*T_short
P_short = P_short/np.sum(P_short, axis=0)
print(P_short, np.sum(P_short, axis=0))

# P_max: Z_max missing -> later
P_max = np.zeros(shape=(lidar_max+1, lidar_max+1))
P_max[-1,:] = 1
print(P_max, np.sum(P_max, axis=0))

# P_rand 
P_rand = np.ones(shape=(lidar_max+1, lidar_max+1))
P_rand[-1,:] = 0
P_rand = P_rand/lidar_max
print(P_rand, np.sum(P_rand, axis=0))

P = z_hit*P_hit + z_short*P_short + z_max*P_max + z_rand*P_rand
print(P, np.sum(P, axis=0))

[[1.00000000e+00 1.92874985e-22 1.38389653e-87]
 [1.92874985e-22 1.00000000e+00 1.92874985e-22]
 [1.38389653e-87 1.92874985e-22 1.00000000e+00]] [1. 1. 1.]
[[1.         0.5621765  0.41922895]
 [0.         0.4378235  0.32649584]
 [0.         0.         0.25427521]] [1. 1. 1.]
[[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]] [1. 1. 1.]
[[0.5 0.5 0.5]
 [0.5 0.5 0.5]
 [0.  0.  0. ]] [1. 1. 1.]
[[0.9625     0.06871765 0.0544229 ]
 [0.0125     0.90628235 0.04514958]
 [0.025      0.025      0.90042752]] [1. 1. 1.]


In [49]:
import numpy as np

# n = 2
# r = np.arange(0, n+1)
# print(r)
# d = np.arange(2, n+1+2)
# o = np.ones(3)

# d=np.outer(d,o)

# r=np.outer(o,r)

# z = d-r
# print(z, z**2)


# print(z[1][0])

n=3
z=np.arange(0,n+1).reshape(-1, 1)
x=np.arange(0,n+1).reshape(+1, -1)
print(np.outer(z, np.ones(n+1)))
M = np.where(z > x, 0, 1)
print(M)

A=np.ones(shape=(4,4))
A=2*A
print(A)
print(M*A)

[[0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [2. 2. 2. 2.]
 [3. 3. 3. 3.]]
[[1 1 1 1]
 [0 1 1 1]
 [0 0 1 1]
 [0 0 0 1]]
[[2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]]
[[2. 2. 2. 2.]
 [0. 2. 2. 2.]
 [0. 0. 2. 2.]
 [0. 0. 0. 2.]]


import numpy a nop

In [13]:
import numpy as np
import logging

# Parameter definition
z_hit = 0.85
z_short = 0.10
z_max = 0.025
z_rand = 0.025
sigma_hit = 0.1
lambda_short = 0.25

# Map and sensor properties
map_resolution = 0.5  # meters per pixel
lidar_max = 2.0       # maximum range in meters
max_range_px = int(lidar_max / map_resolution)

# Logging setup
logging.basicConfig(level=logging.INFO)
logging.info("Precomputing sensor model")

# Normalize sigma and lambda to pixel space
sigma_hit_px = sigma_hit / map_resolution
lambda_short_px = lambda_short / map_resolution

# Table dimensions
table_width = max_range_px + 1
sensor_model_table1 = np.zeros((table_width, table_width))
sensor_model_table2 = np.zeros((table_width, table_width))
vector = True

if vector:

    # Create matrices for d and r
    d_vals = np.arange(table_width).reshape(-1, 1)  # Shape: (table_width, 1)
    r_vals = np.arange(table_width).reshape(1, -1)  # Shape: (1, table_width)

    # Compute z = d - r for Gaussian model
    z = d_vals - r_vals

    # Gaussian distribution (z_hit component)
    gaussian = np.exp(-0.5 * (z**2) / (sigma_hit**2)) / (sigma_hit * np.sqrt(2.0 * np.pi))

    # Exponential distribution (z_short component)
    exponential = np.where(r_vals <= d_vals, lambda_short * np.exp(-lambda_short * r_vals), 0)

    # Normalize the Gaussian and Exponential distributions
    norm_gau = 1.0 / gaussian.sum(axis=1, keepdims=True)  # Shape: (table_width, 1)
    norm_exp = 1.0 / exponential.sum(axis=1, keepdims=True)  # Shape: (table_width, 1)

    # Apply normalizers
    gaussian_normalized = gaussian * norm_gau
    exponential_normalized = exponential * norm_exp

    # Combine into the sensor model table
    sensor_model_table1 = (
        z_hit * gaussian_normalized
        + z_short * exponential_normalized
        + z_rand / table_width
    )
    
if vector: 
    #    Compute normalizers for Gaussian and Exponential distributions
    norm_gau = np.zeros((table_width,))
    norm_exp = np.zeros((table_width,))

    for d in range(table_width):
        sum_gau = 0
        sum_exp = 0
        for r in range(table_width):
            z = float(d - r)
            sum_gau += np.exp(-(z * z) / (2.0 * sigma_hit_px * sigma_hit_px)) / \
                    (sigma_hit_px * np.sqrt(2.0 * np.pi))

            if r <= d:
                sum_exp += lambda_short_px * np.exp(-lambda_short_px * r)

        norm_gau[d] = 1 / sum_gau
        norm_exp[d] = 1 / sum_exp

    # Populate the sensor model table
    for d in range(table_width):
        for r in range(table_width):
            z = float(d - r)
            gaussian = np.exp(-(z * z) / (2.0 * sigma_hit_px * sigma_hit_px)) / \
                    (sigma_hit_px * np.sqrt(2.0 * np.pi))
            exponential = lambda_short_px * np.exp(-lambda_short_px * r) if r <= d else 0

            gaussian_normalized = gaussian * norm_gau[d]
            exponential_normalized = exponential * norm_exp[d]

            sensor_model_table2[d, r] = (
                z_hit * gaussian_normalized +
                z_short * exponential_normalized +
                z_rand / table_width
            )

sensor_model_table = sensor_model_table2-sensor_model_table1
# Display the resulting table
print("Sensor Model Table:")
# print(sensor_model_table1)
# print(sensor_model_table2)
print(sensor_model_table)


INFO:root:Precomputing sensor model


Sensor Model Table:
[[-3.16764339e-06  3.16764339e-06  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 6.03145066e-03 -6.03461829e-03  3.16763159e-06  0.00000000e+00
   0.00000000e+00]
 [ 8.72514394e-03 -1.92682738e-03 -6.80148420e-03  3.16763159e-06
   0.00000000e+00]
 [ 1.05122225e-02  3.47702226e-04 -4.48077185e-03 -6.38232052e-03
   3.16763159e-06]
 [ 1.18633519e-02  1.85473366e-03 -3.03442978e-03 -5.07663749e-03
  -5.60701828e-03]]


In [1]:
def precompute_sensor_model(self):
        '''
        Generate and store a table which represents the sensor model. For each discrete computed
        range value, this provides the probability of measuring any (discrete) range.

        This table is indexed by the sensor model at runtime by discretizing the measurements
        and computed ranges from RangeLibc.
        '''
        print("Precomputing sensor model")
        # sensor model constants
        z_short = self.Z_SHORT
        z_max   = self.Z_MAX
        z_rand  = self.Z_RAND
        z_hit   = self.Z_HIT
        sigma_hit = self.SIGMA_HIT
        
        table_width = int(self.MAX_RANGE_PX) + 1
        self.sensor_model_table = np.zeros((table_width,table_width))

        t = time.time()
        # d is the computed range from RangeLibc
        for d in xrange(table_width):
            norm = 0.0
            sum_unkown = 0.0
            # r is the observed range from the lidar unit
            for r in xrange(table_width):
                prob = 0.0
                z = float(r-d)
                # reflects from the intended object
                prob += z_hit * np.exp(-(z*z)/(2.0*sigma_hit*sigma_hit)) / (sigma_hit * np.sqrt(2.0*np.pi))

                # observed range is less than the predicted range - short reading
                if r < d:
                    prob += 2.0 * z_short * (d - r) / float(d)

                # erroneous max range measurement
                if int(r) == int(self.MAX_RANGE_PX):
                    prob += z_max

                # random measurement
                if r < int(self.MAX_RANGE_PX):
                    prob += z_rand * 1.0/float(self.MAX_RANGE_PX)

                norm += prob
                self.sensor_model_table[int(r),int(d)] = prob

            # normalize
            self.sensor_model_table[:,int(d)] /= norm

        # upload the sensor model to RangeLib for ultra fast resolution
        if self.RANGELIB_VAR > 0:
            self.range_method.set_sensor_model(self.sensor_model_table)

        # code to generate various visualizations of the sensor model
        if False:
            # visualize the sensor model
            fig = plt.figure()
            ax = fig.gca(projection='3d')

            # Make data.
            X = np.arange(0, table_width, 1.0)
            Y = np.arange(0, table_width, 1.0)
            X, Y = np.meshgrid(X, Y)

            # Plot the surface.
            surf = ax.plot_surface(X, Y, self.sensor_model_table, cmap="bone", rstride=2, cstride=2,
                                   linewidth=0, antialiased=True)

            ax.text2D(0.05, 0.95, "Precomputed Sensor Model", transform=ax.transAxes)
            ax.set_xlabel('Ground truth distance (in px)')
            ax.set_ylabel('Measured Distance (in px)')
            ax.set_zlabel('P(Measured Distance | Ground Truth)')

            plt.show()
        elif False:
            plt.imshow(self.sensor_model_table * 255, cmap="gray")
            plt.show()
        elif False:
            plt.plot(self.sensor_model_table[:,140])
            plt.plot([139,139],[0.0,0.08], label="test")
            plt.ylim(0.0, 0.08)
            plt.xlabel("Measured Distance (in px)")
            plt.ylabel("P(Measured Distance | Ground Truth Distance = 140px)")
            plt.show()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (4139417132.py, line 9)